In [ ]:
import numpy as np
class EcoSystem():
    """EcoSystem - vanilla GA algorithm"""

    def __init__(self, de_lege_naturae):

        self.de_lege_naturae = de_lege_naturae

    def evolve(self, epochs):
        """Starting optimization cycle"""
        
        self.ecosystem = []
        self.population_solutions = []
        
        # Imbueing EcoSystem with life        
        for i in xrange(self.de_lege_naturae.ECOSYSTEM_SIZE):
            
            self.ecosystem.append(
                Population(self.de_lege_naturae)
            )
            
            self.ecosystem[-1].grow_population()

            for epoch in xrange(epochs):
                
                self.ecosystem[-1].evaluate_population()
                self.ecosystem[-1].selection()
                self.ecosystem[-1].crossover()
                self.ecosystem[-1].mutation()
                self.ecosystem[-1].elite_preservation()
                
            self.ecosystem[-1].evaluate_population()
            self.population_solutions.append([
                    self.ecosystem[i].fittest_genome_score,
                    self.ecosystem[i].fittest_genome
                ])


class De_lege_naturae():
    """Class encapsulates algorithm's paramaters"""

    def __init__(self, 
                 ecosystem_size, 
                 population_size, 
                 chromosome_sizes,
                 evaluator,
                 tournament_coef = 2,
                 crossover_coef = [1],
                 mutation_coef = [0.01],
                 elitism_coef = 0.05):

        # Input tests
        assert(population_size % 2 == 0)
        
        # Creating EcoSystem
        self.ECOSYSTEM_SIZE = ecosystem_size
        self.POPULATION_SIZE = population_size
        self.CHROMOSOME_SIZES = chromosome_sizes
        self.EVALUATOR = evaluator
        self.TOURNAMENT_COEF = tournament_coef
        self.CROSSOVER_COEF = crossover_coef
        self.MUTATION_COEF = mutation_coef
        self.ELITISM_COEF = elitism_coef
        self.ELITES_NUM = np.round(elitism_coef * population_size).astype(int)
            

class Population():
    """Class encapsualtes population's information and behaviour"""

    def __init__(self, de_lege_naturae):

        self.de_lege_naturae = de_lege_naturae

        # Creating populations        
        
        self.scores = np.zeros(np.sum(self.de_lege_naturae.CHROMOSOME_SIZES))
        
        self.genome = []
        self.elite_genome = []
        self.fittest_genome = [None] * len(self.de_lege_naturae.CHROMOSOME_SIZES)
        
    def grow_population(self):
        """Initializing population with random binary data"""

        for i in xrange(len(self.de_lege_naturae.CHROMOSOME_SIZES)):
            self.genome.append(
                np.random.randint(
                    0,
                    2,
                    size = (self.de_lege_naturae.POPULATION_SIZE, self.de_lege_naturae.CHROMOSOME_SIZES[i])
                )
            )
            self.elite_genome.append(
                np.empty((
                        self.de_lege_naturae.ELITES_NUM,
                        self.de_lege_naturae.CHROMOSOME_SIZES[i]
                    ))
            )
            
    def evaluate_population(self):
        """Population evaluation"""

        self.scores = self.de_lege_naturae.EVALUATOR(self.genome)
        
        fittest_genome_index = np.argmin(self.scores)
        self.fittest_genome_score = self.scores[fittest_genome_index]
        
        for i in xrange(len(self.genome)):
            self.fittest_genome[i] = self.genome[i][fittest_genome_index, :]
            
        elite_indices = np.argpartition(
            self.scores,
            self.de_lege_naturae.ELITES_NUM
        )[: self.de_lege_naturae.ELITES_NUM]
        
        for i in xrange(len(self.genome)):
            self.elite_genome[i] = np.take(self.genome[i], elite_indices, axis = 0)
        
    def selection(self):
        """Selection process implemented as n-member tournament"""

        tournament_selector = np.random.randint(
            0,
            self.de_lege_naturae.POPULATION_SIZE,
            (self.de_lege_naturae.TOURNAMENT_COEF, self.de_lege_naturae.POPULATION_SIZE)
        )
        
        # Tournament
        winner_selector_indices = np.argmin(np.take(self.scores, tournament_selector), axis = 0)
        winner_indices = np.choose(winner_selector_indices, tournament_selector)
        
        for chromosome in self.genome:            
            chromosome[:, :] = np.take(chromosome, winner_indices, axis = 0)
            
    def crossover(self):
        """Sexual recombination implemented as uniform crossover operator"""

        for i in xrange(len(self.genome)):
            
            index_mid = self.genome[i].shape[0] / 2
            
            crossover_mask = np.random.binomial(
                1,
                self.de_lege_naturae.CROSSOVER_COEF[i],
                index_mid
            ).astype(bool)

            gene_selector = np.random.binomial(
                1,
                0.5,
                (np.count_nonzero(crossover_mask), self.genome[i].shape[1])
            ).astype(bool)

            mask = np.zeros((index_mid, gene_selector.shape[1]), dtype=bool)
            mask[crossover_mask, :] = gene_selector

            self.genome[i][:index_mid,:][mask], self.genome[i][index_mid:,:][mask] =\
                self.genome[i][index_mid:,:][mask], self.genome[i][:index_mid,:][mask].copy()
            
    def mutation(self):
        """Mutation processs implemented as uniform mutation"""

        for i in xrange(len(self.genome)):
            mutation_selector = np.random.binomial(
                1,
                self.de_lege_naturae.MUTATION_COEF[i],
                self.genome[i].shape
            )
            self.genome[i] = np.bitwise_xor(self.genome[i], mutation_selector)

    def elite_preservation(self):
        """Elites taking their rightful place"""

        for i in xrange(len(self.genome)):
            self.genome[i][: self.de_lege_naturae.ELITES_NUM, :] = self.elite_genome[i]